In [10]:
import numpy as np
import tensorflow as tf
# import matplotlib.pyplot as plt

from PIL import Image

# ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.models import Model

from keras.layers import Dropout, Flatten, Dense, Conv2D, Input
from keras.layers import GlobalAveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

# from sklearn.metrics import confusion_matrix

Num GPUs Available:  2


In [11]:
physical_devices = tf.config.list_physical_devices('GPU')
for i, device in enumerate(physical_devices):
    print(f"GPU {i}: {device}")


GPU 0: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU 1: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [12]:
path = 'car_make_images/'
training_path = path + 'train'
testing_path = path + 'test'
validation_path = path + 'val'

In [16]:
# Inicializamos los ImageDataGenerator

training_data_generator = ImageDataGenerator(rescale = 1./255,
                              rotation_range = 359,
                              shear_range = 0.2,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              zoom_range = 0.2,
                              horizontal_flip = True,
                              vertical_flip = True,
                              preprocessing_function = None)

validation_data_generator = ImageDataGenerator(rescale = 1./255)  # Removed augmentation
test_data_generator = ImageDataGenerator(rescale = 1./255)  # Removed augmentation

In [17]:
# Entrenamos los ImageDataGenerator

# Para poder usar .flow_from_directory(), necesitamos tener las carpetas ordenadas, es decir:

# una carpeta de train con una carpeta de imagenes para cada clase
# una carpeta de validation con una carpeta de imagenes para cada clase
# y una carpeta de test con una carpeta de imagenes para cada clase

size = 200
batch_size = 30  # For example
num_classes = 39  # Change to the number of classes you actually have

training_generator = training_data_generator.flow_from_directory(training_path,
                                                                 target_size = (size, size),
                                                                 batch_size = 30,
                                                                 class_mode = "categorical")

validation_generator = validation_data_generator.flow_from_directory(validation_path,
                                                                     target_size = (size, size),
                                                                     batch_size = 1,
                                                                     class_mode = "categorical")

test_generator = test_data_generator.flow_from_directory(testing_path,
                                                         target_size = (size, size),
                                                         batch_size = 1,
                                                         class_mode = "categorical")

# En caso de que la clasificación sea multi-clase, el cambiamos el parámetro class_mode = "categorical"

Found 11573 images belonging to 39 classes.
Found 2813 images belonging to 39 classes.
Found 2871 images belonging to 39 classes.


In [26]:
with tf.device('/GPU:0'):

    model = Sequential()

    model.add(Input(shape = (size, size, 3)))

    model.add(Conv2D(filters = 16, kernel_size = 9, padding = "same", activation = "relu"))
    model.add(MaxPooling2D(pool_size = 2))

    model.add(Conv2D(filters = 16, kernel_size = 9, padding = "same", activation = "relu"))
    model.add(MaxPooling2D(pool_size = 2))

    model.add(Flatten())

    model.add(Dense(units = 120, activation = "relu"))
    model.add(Dense(units = 120, activation = "relu"))
    model.add(Dense(units = 120, activation = "relu"))
    model.add(Dropout(0.15))
    model.add(Dense(units = num_classes, activation = "softmax"))  # Changed to 'softmax'
    
    # Model Compilation
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

    # Model Training
    history = model.fit(training_generator, epochs = 150, validation_data = validation_generator)

386/386 [==============================] - 134s 345ms/step - loss: 3.6435 - accuracy: 0.0397 - val_loss: 3.6140 - val_accuracy: 0.0462
